In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows

In [3]:
cd D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_chunking

D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_chunking


c:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
train_path = r"Data\Raw_data\train.csv"
dev_path = r"Data\Raw_data\dev.csv"
test_path = r"Data\Raw_data\test.csv"

In [5]:
from Code.Preprocessing import unicode, dupplicate_punctuation, deleteIcon
import pandas as pd
import numpy as np

classes = ['Constructiveness','Toxicity']

def load_data(path_csv):
    data = pd.read_csv(path_csv, encoding = 'utf-8')
    # Apply the replacement function to the 'content' column
    data['Sentence'] = data['content'].apply(unicode)
    data['Sentence'] = data['Sentence'].apply(lambda row: dupplicate_punctuation(row)[0])
    data['Text'] = data['Sentence'].apply(deleteIcon)
    data['Tag'] = data['index_spans'].apply(literal_eval)
    data['Tag'] = data['Tag'].apply(lambda row: 0.0 if row == [] else 1.0)
    data = data[['Text', 'Tag']]

    return data

In [6]:
test = load_data(test_path)
train = load_data(train_path)
dev = load_data(dev_path)

test.head()

,Text,Tag
0,anh bar .,0.0
1,hello thầy,0.0
2,từ lúc mấy bro cmt cực kì cl gì đấy là rất khó để ngó được cái cmt hay buồn thực sự,1.0
3,nghe trần dần thấy cười đau cả bụng,0.0
4,idol tui có khác. phải v chứ ai như cái con phò tiệp tiệp j đó. đm nó như con đàn bà.,1.0


In [7]:
train.reset_index(inplace=True)
dev.reset_index(inplace=True)
test.reset_index(inplace=True)

test.head()

,index,Text,Tag
0,0,anh bar .,0.0
1,1,hello thầy,0.0
2,2,từ lúc mấy bro cmt cực kì cl gì đấy là rất khó để ngó được cái cmt hay buồn thực sự,1.0
3,3,nghe trần dần thấy cười đau cả bụng,0.0
4,4,idol tui có khác. phải v chứ ai như cái con phò tiệp tiệp j đó. đm nó như con đàn bà.,1.0


In [9]:
train['sentence_id'] = train['index']
dev['sentence_id'] = dev['index']
test['sentence_id'] = test['index']
test.head()

,index,Text,Tag,sentence_id
0,0,anh bar .,0.0,0
1,1,hello thầy,0.0,1
2,2,từ lúc mấy bro cmt cực kì cl gì đấy là rất khó để ngó được cái cmt hay buồn thực sự,1.0,2
3,3,nghe trần dần thấy cười đau cả bụng,0.0,3
4,4,idol tui có khác. phải v chứ ai như cái con phò tiệp tiệp j đó. đm nó như con đàn bà.,1.0,4


In [10]:
# from vncorenlp import VnCoreNLP
# # Initialize VNCoreNLP client (adjust the path as necessary)
# rdrsegmenter = VnCoreNLP(address='http://127.0.0.1', port=9000)

from vncorenlp import VnCoreNLP

# Ensure that the JAR file path is correct
rdrsegmenter = VnCoreNLP(r"D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_tokenize\Code\VnCoreNLP\VnCoreNLP-1.1.1.jar", 
                         annotators="wseg", max_heap_size='-Xmx500m')

# Define a function to process text with VNCoreNLP
def process_text_vncorenlp(text):
    try:
        # Perform word segmentation
        segmented_text = rdrsegmenter.tokenize(text)
        # Flatten the list of tokenized sentences into a single string
        return ' '.join([' '.join(sentence) for sentence in segmented_text])
    except Exception as e:
        print(f"Error processing text: {e}")
        return text


In [11]:

# Apply the VNCoreNLP processing to a specific column (e.g., 'text')
test['Text'] = test['Text'].apply(process_text_vncorenlp)
train['Text'] = train['Text'].apply(process_text_vncorenlp)
dev['Text'] = dev['Text'].apply(process_text_vncorenlp)
test.head()

,index,Text,Tag,sentence_id
0,0,anh bar .,0.0,0
1,1,hello thầy,0.0,1
2,2,từ lúc mấy bro cmt cực_kì cl gì đấy là rất khó để ngó được cái cmt hay buồn thực_sự,1.0,2
3,3,nghe trần dần thấy cười đau cả bụng,0.0,3
4,4,idol tui có khác . phải v chứ ai như cái con phò tiệp tiệp j đó . đm nó như con đàn_bà .,1.0,4


In [12]:
import os
folder_path = "Data"
if not os.path.exists(folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(folder_path)

train.to_csv(os.path.join(folder_path, "train.csv"), index=False)
dev.to_csv(os.path.join(folder_path, "dev.csv"), index=False)
test.to_csv(os.path.join(folder_path, "test.csv"), index=False)